In [1]:
using LinearAlgebra
using QuantumOptics
using WignerSymbols
using Plots
plotly()
using Printf 
import QuantumOptics.steadystate as steady

In [57]:
nucI=3//2
grJ=1//2
exJ=1//2

exFmax=nucI+exJ
exFmin=abs(nucI-exJ)
grFmax=nucI+grJ
grFmin=abs(nucI-grJ)

grStates=Dict([((2,2),1),((2,1),2),((2,0),3),((2,-1),4),((2,-2),5),((1,1),6),((1,0),7),((1,-1),8)])
exStates=Dict([((2,2),9),((2,1),10),((2,0),11),((2,-1),12),((2,-2),13),((1,1),14),((1,0),15),((1,-1),16)])
rbD1=NLevelBasis(16)


# Function to return transition operator from initial state |F1,mF> to final state |F2,mF+q>
function sigma(theBasis,initialState,finalState,F1,F2,q,mF)
    transition(theBasis,initialState[(F1,mF),finalState(F2,mF+q)]) 
end


# Function to return dipole matrix element for transition from |J1,nucI,F1,mF> to |J1,nucI,F2,mF> for photon
# with polarization q.
function dipole(J1,J2,nucI,F1,F2,q,mF)
    wigner3j(F1,1,F2,mF,q,-mF-q)*wigner6j(J1,J2,1,F2,F1,nucI)*(-1)^(mF+J1+nucI)*sqrt((2F1+1)*(2*F2+1)*(2*J1+1))
end 


σ=[] # empty array for transition operators
dip=[]  # empty array for dipole matrix elements <F1,mF|er_q|F',mF'> (see Steck, Rb87, eq. (35))

F1vals=[1,2]   # possible ground state F values
F2vals=[1,2]   # possible excited state F values
qvals=[-1,1]   # possible photon polarizations (sigma plus and sigma -. 0 would be linear)

n2Fm=Dict{Int64,Tuple}()     # empty Dictionary "n to ("ground/excited",(F,m))"
Fm2n=Dict{Tuple,Int64}()     # opposite direction "(F,m) to n", i.e., (("ground/excited",(F,m)) are keys
myIndex=1

# Fill dictionary with values
for F=grFmax:-1:grFmin
    for m=F:-1:-F
        get!(n2Fm,myIndex,("ground",(F,m)))
        get!(Fm2n,("ground", (F,m)),myIndex)
        myIndex+=1
    end
end

# Fill excited state dictionary 
for F=exFmax:-1:exFmin
    for m=F:-1:-F
        get!(n2Fm,myIndex,("excited", (F,m)))
        get!(Fm2n,("excited", (F,m)),myIndex)
        myIndex+=1
    end
end

# loop over dictionary entries (keys)
for i in keys(n2Fm)
    if n2Fm[i][1]=="ground"
        F1=n2Fm[i][2][1]
        m1=n2Fm[i][2][2]
        for F2 = exFmin:exFmax
            q=+1
            if (newN=get(Fm2n,("excited",(F2,m1+q)),0)) > 0
 #               q=+1
 #               println("σ+: i = ", i, "  newN = ", newN, "  F1 = ", F1, "  m1 = ", m1, "  F2 = ", F2, "  m1+1 = ", (m1+1), " 3j = ", wigner3j(F1,1,F2,m1,q,-m1-q), " 6j = ", wigner6j(exJ,grJ,1,F2,F1,nucI))
                push!(σ,transition(rbD1,i,newN))
                push!(dip,dipole(grJ,exJ,nucI,F1,F2,q,m1))
            end
            q=-1
            if (newN=get(Fm2n,("excited",(F2,m1+q)),0)) > 0
#                q=-1
#                println("σ-: i = ", i, "  newN = ", newN, "  F1 = ", F1, "  m1 = ", m1, "  F2 = ", F2, "  m1-1 = ", (m1-1), " 3j = ", wigner3j(F1,1,F2,m1,q,-m1-q), " 6j = ", wigner6j(exJ,grJ,1,F2,F1,nucI))
                push!(σ,transition(rbD1,i,newN))
                push!(dip,dipole(grJ,exJ,nucI,F1,F2,q,m1))
            end
        end
    end
end


In [58]:
dip

24-element Vector{Any}:
  0.7071067811865475
  0.4082482904638631
 -0.28867513459481287
 -0.28867513459481287
 -0.7071067811865475
  0.7071067811865475
 -0.4082482904638631
  0.28867513459481287
 -0.7071067811865475
 -0.28867513459481287
  0.28867513459481287
  0.28867513459481287
  0.5
 -0.5
 -0.28867513459481287
  0.28867513459481287
 -0.49999999999999994
 -0.49999999999999994
  0.49999999999999994
  0.5
 -0.4082482904638631
  0.49999999999999994
  0.4082482904638631
 -0.5

In [61]:
length(σ)

24

In [62]:
hfsEx=814.5               # excited states hyperfine splitting
hfsGr=6834.682610904290   # ground state hyperfine splitting
grF1=-0.625*hfsGr         # distance from "center of mass to ground F=1"
grF2=0.375*hfsGr          # distance from "center of mass to ground F=2"
exF1=-0.625*hfsEx         # distance from "center of mass to excited F=1"
exF2=0.375*hfsEx          # distance from "center of mass to ground F=2"
hfsE=Dict(  ("ground",(1,1))=>grF1,
            ("ground",(1,0))=>grF1, 
            ("ground",(1,-1))=>grF1,
            ("ground",(2,2))=>grF2, 
            ("ground",(2,1))=>grF2,
            ("ground",(2,0))=>grF2,
            ("ground",(2,-1))=>grF2,
            ("ground",(2,-2))=>grF2,
            ("excited",(1,1))=>exF1,
            ("excited",(1,0))=>exF1, 
            ("excited",(1,-1))=>exF1,
            ("excited",(2,2))=>exF2, 
            ("excited",(2,1))=>exF2,
            ("excited",(2,0))=>exF2,
            ("excited",(2,-1))=>exF2,
            ("excited",(2,-2))=>exF2,
            )
ωLGr1=-7000  # GHz/Tesla   Larmor frequency
ωLGr2=7000 
ωLEx1=-2300  # GHz/Tesla   Larmor frequency
ωLEx2=2300   
larmor=Dict(("ground",(1,1))=>1*ωLGr1,
            ("ground",(1,0))=>0, 
            ("ground",(1,-1))=>(-1)*ωLGr1,
            ("ground",(2,2))=>2*ωLGr2, 
            ("ground",(2,1))=>1*ωLGr2,
            ("ground",(2,0))=>0,
            ("ground",(2,-1))=>(-1)*ωLGr2,
            ("ground",(2,-2))=>(-2)*ωLGr2,
            ("excited",(1,1))=>1*ωLEx1,
            ("excited",(1,0))=>0, 
            ("excited",(1,-1))=>(-1)*ωLEx1,
            ("excited",(2,2))=>2*ωLEx2, 
            ("excited",(2,1))=>1*ωLEx2,
            ("excited",(2,0))=>0,
            ("excited",(2,-1))=>(-1)*ωLEx2,
            ("excited",(2,-2))=>(-2)*ωLEx2,
            )
Ω=1.0       # Rabi frequency
Γ=1.0       # Natual linewidth
γ=1.0       # Transit relaxation

gamma=Dict(("ground",(1,1))=>γ,
            ("ground",(1,0))=>γ, 
            ("ground",(1,-1))=>γ,
            ("ground",(2,2))=>γ, 
            ("ground",(2,1))=>γ,
            ("ground",(2,0))=>γ,
            ("ground",(2,-1))=>γ,
            ("ground",(2,-2))=>γ,
            ("excited",(1,1))=>γ+Γ,
            ("excited",(1,0))=>γ+Γ, 
            ("excited",(1,-1))=>γ+Γ,
            ("excited",(2,2))=>γ+Γ, 
            ("excited",(2,1))=>γ+Γ,
            ("excited",(2,0))=>γ+Γ,
            ("excited",(2,-1))=>γ+Γ,
            ("excited",(2,-2))=>γ+Γ,
            )

Δ=100       # Detuning from D1 transition at 794.978851156 nm (377.107464380 THz)

proj=[]
detune=[]
ωL=[]
H=identityoperator(σ[1])*0
#J=identityoperator(σ[1])*0
J=[]
absorption=[]
for B=0:1e-5:0.001
    for i=1:length(n2Fm)
        push!(proj,transition(rbD1,i,i))
        push!(ωL,larmor[n2Fm[i]])
        push!(detune,hfsE[n2Fm[i]]-Δ)
        H += ( detune[i] + ωL[i]*B )*proj[i]    # Hamiltonian
        for k=1:length(σ)
            H+= Ω*dip[k]*(σ[k]*dagger(σ[k]))
        end
        push!(J,sqrt(gamma[n2Fm[i]])*proj[i])                                   # Jump operator
        for j=1:length(n2Fm)
            if n2Fm[i][1]=="excited" && n2Fm[j][1]=="ground"
                push!(J,sqrt(Γ)*transition(rbD1,i,j))
            end
        end
    end
# Initial state
    ϕ₀ = nlevelstate(rbD1,1)⊗dagger(nlevelstate(rbD1,1))
    t_out, ρ_master = steady.master(H,J)
    expect_val=0.0
    for k in 1:length(σ) 
        expect_val = real(expect(σ[k]*dagger(σ[k]), ρ_master))
    end
    push!(absorption,expect_val[1][1])
end

#ϕ₀ = nlevelstate(rbD1,4)⊗dagger(nlevelstate(rbD1,4))



In [63]:
Bseries=range(0,0.001,step=1e-5)
plot(Bseries,absorption)

<!DOCTYPE html>
 
 
 Plots.jl

In [64]:
ρ_master

2-element Vector{Operator{NLevelBasis{Int64}, NLevelBasis{Int64}, Matrix{ComplexF64}}}:
 Operator(dim=16x16)
  basis: NLevel(N=16)
 1.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+

In [9]:
ϕ₀ = nlevelstate(rbD1,1)⊗dagger(nlevelstate(rbD1,1))

Operator(dim=16x16)
  basis: NLevel(N=16)
 1.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.

In [11]:
H

Operator(dim=16x16)
  basis: NLevel(N=16)
⠑⢄⠀⠀⠀⠀⠀⠀
⠀⠀⠑⢄⠀⠀⠀⠀
⠀⠀⠀⠀⠑⢄⠀⠀
⠀⠀⠀⠀⠀⠀⠑⢄

In [12]:
import QuantumOptics.steadystate as steady 
t_out, ρ_master = steady.master(H,J)

([0.0, 1.262376595100603], Operator{NLevelBasis{Int64}, NLevelBasis{Int64}, Matrix{ComplexF64}}[Operator(dim=16x16)
  basis: NLevel(N=16)
 1.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  

In [6]:
n2Fm[1]

("ground", (2//1, 2//1))

In [7]:
n2Fm[1][1]

"ground"